**Twitter Sentiment Analysis Project Using Machine Learning Algorithms**

In [1]:
# Installing kaggle library
! pip install kaggle

**Connecting Kaggle API through kaggle.json file**

**Importing Twitter Sentiment Analysis Dataset**

In [5]:
# Importing dataset using kaggle API
# Dataset link: kaggle.com/datasets/kazanova/sentiment140/data
!kaggle datasets download -d kazanova/sentiment140


  0%|          | 0.00/80.9M [00:00<?, ?B/s]
  1%|          | 1.00M/80.9M [00:01<02:03, 679kB/s]
  2%|▏         | 2.00M/80.9M [00:01<01:00, 1.36MB/s]
  4%|▎         | 3.00M/80.9M [00:01<00:37, 2.20MB/s]
  5%|▍         | 4.00M/80.9M [00:02<00:25, 3.18MB/s]
  7%|▋         | 6.00M/80.9M [00:02<00:15, 5.17MB/s]
 10%|▉         | 8.00M/80.9M [00:02<00:11, 6.76MB/s]
 12%|█▏        | 10.0M/80.9M [00:02<00:09, 8.09MB/s]
 15%|█▍        | 12.0M/80.9M [00:02<00:08, 9.01MB/s]
 17%|█▋        | 14.0M/80.9M [00:02<00:07, 9.74MB/s]
 20%|█▉        | 16.0M/80.9M [00:03<00:06, 10.3MB/s]
 22%|██▏       | 18.0M/80.9M [00:03<00:06, 10.7MB/s]
 25%|██▍       | 20.0M/80.9M [00:03<00:05, 10.9MB/s]
 27%|██▋       | 22.0M/80.9M [00:03<00:05, 11.1MB/s]
 30%|██▉       | 24.0M/80.9M [00:03<00:05, 11.3MB/s]
 32%|███▏      | 26.0M/80.9M [00:03<00:05, 11.4MB/s]
 35%|███▍      | 28.0M/80.9M [00:04<00:04, 11.5MB/s]
 37%|███▋      | 30.0M/80.9M [00:04<00:04, 11.6MB/s]
 40%|███▉      | 32.0M/80.9M [00:04<00:04, 11.6MB/s]
 4

In [6]:
# Extracting the compressed dataset
from zipfile import ZipFile
dataset = 'sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


**Importing the Dependencies**

In [7]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gychi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# Printing th stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**Data Processing**

In [11]:
# Loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [12]:
# Checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [13]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [15]:
# naming the columns and reading the datasets again

column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', names  = column_names, encoding = 'ISO-8859-1')

In [16]:
# checking the number of rows and columns again
twitter_data.shape

(1600000, 6)

In [17]:
# printing the first 5 rows of the dataframe again
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [18]:
# counting the number of missing values in the dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [19]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

**Convert the target label '4' to '1'**

In [20]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [21]:
# checking the distribution of target column again
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

Label Names:

*   0 --> Negative Tweet
*   1 --> Positive Tweet

**Stemming:** process of reducing a word to its Root word
<br>
example: actor, actress, acting = act

In [22]:
port_stem = PorterStemmer()

In [23]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [24]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [25]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [26]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [27]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [28]:
# seperating the data and label from unnecessary content
x = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [29]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [30]:
print(y)

[0 0 0 ... 1 1 1]


**Splitting the dataset into training and testing data**

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

In [32]:
print(x.shape, x_train.shape, x_test.shape)

(1600000,) (1280000,) (320000,)


In [33]:
print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [34]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


**Converting textual data to numeric data**

In [35]:
# using vectorizer
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [36]:
print(x_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [37]:
print(x_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

**Training the *Logistic Regression* Machine Learning Model**

In [38]:
model = LogisticRegression(max_iter=1000)

In [39]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

**Model Evaluation**

In [40]:
# calculating accuracy score on training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train, x_train_prediction)

In [41]:
print("Accuracy score on the training data: ", training_data_accuracy)

Accuracy score on the training data:  0.79871953125


In [42]:
# calculating accuracy score on test data
x_test_prediction = model.predict(x_test)
testing_data_accuracy = accuracy_score(y_test, x_test_prediction)

In [43]:
print("Accuracy score on the testing data: ", testing_data_accuracy)

Accuracy score on the testing data:  0.77668125


Model accuracy = 77.80 %

**Saving the trained model**

In [44]:
import pickle

In [45]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

**Using the saved model for future predictions**

In [46]:
import gradio as gr
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

C:\Users\gychi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
# Load the trained logistic regression model
model = joblib.load('trained_model.sav')

In [50]:
# Load NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gychi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
# Initialize PorterStemmer
port_stem = PorterStemmer()

In [52]:
# Define preprocessing function
def preprocess_text(text):
    # Remove special characters and numbers, convert to lowercase
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    # Tokenize text
    words = text.split()
    # Remove stopwords and perform stemming
    stemmed_words = [port_stem.stem(word) for word in words if not word in stop_words]
    # Join stemmed words back into a single string
    processed_text = ' '.join(stemmed_words)
    return processed_text

In [53]:
# Define sentiment analysis function
def predict_sentiment(text):
    # Preprocess input text
    processed_text = preprocess_text(text)
    # Convert preprocessed text into numeric data using the same vectorizer
    processed_text_numeric = vectorizer.transform([processed_text])
    # Predict sentiment
    sentiment = model.predict(processed_text_numeric)[0]
    return "Positive" if sentiment == 1 else "Negative"

In [55]:
# Create Gradio interface
text_input = gr.Textbox(lines=5, label="Enter your text here")
output_text = gr.Textbox(label="Predicted sentiment")

model_interface = gr.Interface(
    fn=predict_sentiment, 
    inputs=text_input, 
    outputs=output_text, 
    title="Sentiment Analysis", 
    description="Enter a text and predict its sentiment (Positive/Negative)"
)
model_interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://4ba5e1a25dc8f05bc2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
